In [191]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

In [192]:
# open files
test = open("data/rv0.5/test")
rv05 = open("data/rv0.5/initial.collision.log")
rv2 = open("data/rv2.0/initial.collision_rv2.log")
bhcollrv05 = open("data/rv0.5/bh.collosion.log",mode='w')

In [193]:
# read in files line by line and seperate
def extractData(infile,outfile):
    line = infile.readline()
    header = line
    line = infile.readline()
    while line:
        sline = line.split(" ")
        type1 = -1
        type2 = -1
        type3 = -1
        type4 = -1
        for elem in sline[6:]:
            if elem[:5] == "type1":
                type1 = int(elem[6:])
            elif elem[:5] == "type2":
                type2 = int(elem[6:])
            elif elem[:5] == "type3":
                type3 = int(elem[6:])
            elif elem[:5] == "type4":
                type4 = int(elem[6:])
        if type1 == 14 or type2 == 14 or type3 ==14 or type4 == 14:
            outfile.write(line)
        line = infile.readline()

In [194]:
extractData(rv05,bhcollrv05)

In [195]:
# close files
test.close()
rv05.close()
rv2.close()
bhcollrv05.close()